In [ ]:
import rosbag
from rosbag import Bag
import os
import sys

In [ ]:
####################################################
########## Modify the Configuration  Here ##########
####################################################

#Specify the path where the bags are stored.
main_path = r"./2023-11-03-11-07-11"

# Define a dictionary where the key is the bag file and the value is a list of topics to merge
bags_and_topics = {
    'reference.bag': ['/novatel/reference/postprocessed/inspva','/novatel/reference/postprocessed/inspvax'],
    'imus.bag': ['/xsens/imu', '/obd2/speed'],
    'gnss.bag': ['/xsens/gnss'],
    # Add more bags and their corresponding topics here
}


# Path of the new bag file
output_bag_path = os.path.join(main_path,'merged.bag')

In [ ]:
#Helper functions
# Function to check if all bags are available
def check_bags_exist(main_path,bag_files):
    missing_bags = [bag for bag in bag_files if not os.path.exists(os.path.join(main_path,bag))]
    if missing_bags:
        print(f"Error: The following bag files are missing: {missing_bags}")
        return False
    return True

# Function to check if each bag contains the specified topics
def check_topics_in_bags(main_path,bags_and_topics):
    for bag_file, topics in bags_and_topics.items():
        bag_file_path = os.path.join(main_path,bag_file)
        with Bag(bag_file_path, 'r') as bag:
            available_topics = bag.get_type_and_topic_info()[1].keys()
            missing_topics = [topic for topic in topics if topic not in available_topics]
            if missing_topics:
                print(f"Error: The following topics are missing in {bag_file}: {missing_topics}")
                return False
    return True

In [ ]:
# Perform some checks
if check_bags_exist(main_path,bags_and_topics.keys()) and check_topics_in_bags(main_path,bags_and_topics):
    print("Bags and Topics are available")

In [ ]:
# Open a new bag file to write the merged data
with Bag(output_bag_path, 'w') as outbag:
    for bag_file, topics_to_merge in bags_and_topics.items():
        bag_file_path = os.path.join(main_path,bag_file)
        
        # Open each bag file
        with Bag(bag_file_path, 'r') as inbag:
            print("\n----------------------------------------")
            #bag timing info
            start_time = inbag.get_start_time()
            end_time = inbag.get_end_time()
            total_duration = end_time - start_time
            last_time = start_time

            for topic, msg, t in inbag.read_messages():
                # Calculate progress
                current_time = t.to_sec()
                elapsed_time = current_time - start_time
                progress = (elapsed_time / total_duration) * 100

                # Print progress Every second
                if current_time - last_time >= 1: 
                    sys.stdout.write(f"\rProcessing {bag_file}: {progress:.2f}% complete.")
                    sys.stdout.flush()
                    last_time = current_time
                    
                if topic in topics_to_merge:
                    outbag.write(topic, msg, t)

        sys.stdout.write(f"\rProcessing bag: 100.0% complete")
        sys.stdout.flush()


print("\n----------------------------------------")
print("Finished!")